## Setup spark and install modules

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, current_timestamp

In [2]:
spark = SparkSession.builder.appName("LogsAnalysis").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/04/08 08:56:28 WARN Utils: Your hostname, DELEQ0283302041 resolves to a loopback address: 127.0.1.1; using 172.31.227.62 instead (on interface eth0)
24/04/08 08:56:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/08 08:56:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/08 08:56:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Setup


### Generate data
For out practice purposes, we will generate fake Apache Access logs.
>1. Open terminal and change directory to "Data" folder
>2. In terminal, run command `python log_generator apache_access_schema.yaml`, add flag `-t` to truncate previously created logs
>3. Based on provide .yaml file, new `log.txt.{id}` file should appear every 5s with 50 lines of fake logs.

## Practice

1. Create DataFrame representing the stream of raw log data lines ariving to `/'log/apache_access` directory

In [3]:
access_lines = spark.readStream.text("../../mock_logs/log/apache_access")

2. Parse log data to a DataFrame

In [4]:
# Parse out the common log format to a DataFrame
contentSizeExp = r'\s(\d+)$'
statusExp = r'\s(\d{3})\s'
generalExp = r'\"(\S+)\s(\S+)\s*(\S*)\"'
timeExp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2}:\d{4})]'
hostExp = r'(^\S+\.[\S+\.]+\S+)\s'

logsDF = access_lines.select(regexp_extract('value', hostExp, 1).alias('host'),
                        regexp_extract('value', timeExp, 1).alias('timestamp'),
                        regexp_extract('value', generalExp, 1).alias('method'),
                        regexp_extract('value', generalExp, 2).alias('endpoint'),
                        regexp_extract('value', generalExp, 3).alias('protocol'),
                        regexp_extract('value', statusExp, 1).cast('integer').alias('status'),
                        regexp_extract('value', contentSizeExp, 1).cast('integer').alias('content_size'))

3. Add event time column with current time for window aggregation

In [ ]:
LogsEventimeDF = logsDF.withColumn("eventTime", current_timestamp())

4. Define windowDuration adn slideDuration

In [ ]:
windowDuration = '2 minutes'
slideDuration = '30 seconds'

5. Classroom cleanup
IMPORTANT: Kill running log_generator script in terminal

In [11]:
query.stop()
spark.stop()